In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
import seaborn as sns

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA, TruncatedSVD, SparsePCA
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, hamming_loss, f1_score

from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [13]:
X_test = pd.read_csv('../Data/X_test_clean.csv')
y_train = pd.read_csv('../Data/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Labels.csv')
X_train = pd.read_csv('../Data/X_train_clean.csv')
X_test_orig = pd.read_csv('../Data/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Test_Features.csv')

In [14]:
labels = ['h1n1_vaccine', 'seasonal_vaccine']


In [16]:
y_train_enc =  2 * y_train['h1n1_vaccine'].values + y_train['seasonal_vaccine'].values

model = CatBoostClassifier(
    loss_function='MultiClass',
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg = 9,
    rsm=0.5,
    bagging_temperature=3,
    verbose=0
)

model.fit(X_train, y_train_enc)


y_pred_enc = model.predict(X_test).astype(int)

h1n1_pred   = y_pred_enc // 2
seasonal_pred = y_pred_enc %  2

y_pred = pd.DataFrame({
    'h1n1_vaccine':   h1n1_pred.flatten(),
    'seasonal_vaccine': seasonal_pred.flatten()
}, index=X_test_orig.index) 

y_pred

,h1n1_vaccine,seasonal_vaccine
0,0,0
1,0,0
2,1,1
3,0,1
4,0,0
...,...,...
26703,0,0
26704,0,0
26705,0,0
26706,0,0


In [29]:
proba_list = model.predict_proba(X_test)
P = np.vstack(proba_list) 
p_h1n1     = P[:, 2] + P[:, 3]
p_seasonal = P[:, 1] + P[:, 3]

y_pred_proba = pd.DataFrame({
    'h1n1_vaccine':     np.round(p_h1n1,1),
    'seasonal_vaccine': np.round(p_seasonal,1)
}, index=np.arange(26707,53415))

y_pred_proba.to_csv("../data/submission1.csv")

In [23]:
X_test_orig.index

RangeIndex(start=0, stop=26708, step=1)